In [1]:
from functools import partial
import sys
sys.path.insert(0, '/Users/wakita/Dropbox (smartnova)/lib/python3/sympy')
#sys.path.insert(0, '/Users/wakita/db-smartnova.bak/lib/python3/sympy')

import sympy as sp
from sympy.utilities.lambdify import lambdify, lambdastr

import numpy as np
np.seterr(all='raise')

from scipy.optimize import minimize

from nbsupport import md
from snsympy import *

ImportError: SymPy now depends on mpmath as an external library. See http://docs.sympy.org/latest/install.html#mpmath for more information.

# D-dimensional KK method

## Dimension

In [ ]:
dim = 2

The symbol `dim` represents the dimension of the visualization.  Normally its value is either 2 or 3 but a larger value can be given.

## Graph

Consider a graph $G = (V, E)$ that consists form $n$ vertices ($|V| = n$).

In [ ]:
n = sp.Symbol('n', integer=True)

## The spring system

The KK spring model assumes that every pair of vertices $(v_i, v_j)$ is connected by a spring whose natural length and strength are $L_{i, j}$ and $K_{i, j}$, respectively.

KK method picks up a vertex $p$ and attempts to relocate it, while keeping other vertices stationary, trying to minimize the spring potential.  By repeating this process, the spring potential for the whole spring system is minimized.

To keep things simple, we consider springs connected to a vertex $v_i \in V$.  Let $v_j \in P \setminus \{v_i\}$ be a vertex other than $v_i$ in the graph, location function $P: (P \setminus \{v_i\}) \rightarrow R^{\mathrm {dim}}$ denotes the location of the vertex (other than $v_i$), $p$ be the current location of $v_i$, and that $L_j$ and that $K_j$ give the natural length and the strength of the spring that connects $v_i$ and $v_j$.

In [ ]:
ni = n-1
Pi = sp.IndexedBase('P', shape=(dim, ni))
Ki = sp.IndexedBase('K')
Li = sp.IndexedBase('L')

p = sp.IndexedBase('p')

j, d = [sp.Idx(*spec) for spec in [('j', ni), ('d', dim)]]
j_range, d_range = [(idx, idx.lower, idx.upper) for idx in [j, d]]

For convenience, we introduce a notation $q(d)$ that represents the $d$'th element of the location of $v_j$, namely $P_{d, j}$.

In [ ]:
def q(d): return Pi[d, j]

### Actual length of a spring that connects $v_i$ and $v_j$

In [ ]:
LENGTH_J = sp.Symbol('Length_j')

LengthJ = sp.sqrt(sp.Sum((p[d] - q(d)) ** 2, d_range)).doit()

In [ ]:
md('The length of the spring that connects $v_i$ and $v_j$ is the distance between $p$ and $q$: $$',
   LENGTH_J, '=', LengthJ, '$$.')

### Potential energy stored in the spring system

In [ ]:
POTENTIAL_J, POTENTIAL = sp.var('Potential_j, Potential')

PotentialJ = Ki[j] * (LengthJ - Li[j]) ** 2
Potential = sp.Sum(PotentialJ, j_range)

In [ ]:
md(r'Potential energy for the spring that connects $p \text{ and }q$: $',
   POTENTIAL_J, '=', Ki[j] * (LENGTH_J - Li[j]), '=', PotentialJ, '$')

md('Collective potential energy of all the springs that connect to $p$: $$',
   POTENTIAL, '=', sp.Sum(POTENTIAL_J, j_range), '=', Potential, '$$')

## Code generation

A python function that implements a mathematical formula is obtained by passing the formula to the `lambdify` a list of free variable names and the formula.  The python function is a lambda form that takes actual parameters that correspond to the free variables and (numerically) computes the formula.

Using `lambdify`, we can easily obtain a Python function `f_potential` that computes the spring potential energy ($\mathit {Potential}$).

In [ ]:
Params = [n, Ki, Li, Pi, p]

potential_j = lambdify((*Params, j), PotentialJ, dummify=False)
potential = lambdify(Params, Potential, dummify=False)

In [ ]:
md('The implementation of the math formula $', PotentialJ, '$ in Python function is the following:')
md('```\n', lambdastr((*Params, j), PotentialJ), '\n```')

md('The implementation of the math formula $', Potential, '$ in Python function is the following:')
md('\n\n```\n', lambdastr(Params, Potential), '\n```')

In [ ]:
def testcase(P, K, L, i):
    n = P.shape[1]
    Ki = K[:,i]
    Li = L[:,i]
    p  = P[:,i]
    Pi = P.take(list(range(i)) + list(range(i+1, n)), axis=1)
    return [n, Ki, Li, Pi, p]

def testcase1():
    P = np.eye(dim, dim+1) * 5
    n = P.shape[1]
    K = L = np.ones((n, n))
    return testcase(P, K, L, dim)

def test_potential():
    md('## Tests')
    md('### Potential test')
    args = testcase1()
    Pi = args[3]
    
    for j in range(Pi.shape[1]):
        md('- $', POTENTIAL, '_', j, '=', potential_j(*args, j), '$')

test_potential()

## Derivative Formulae

In [ ]:
X = [p[i] for i in range(dim)]
PotentialDerivatives, potential_derivatives = derivatives(Potential, X, Params)

### Function

In [ ]:
md('$$', PotentialDerivatives[0], '$$')

### Gradient

In [ ]:
md('$$', PotentialDerivatives[1], '$$')

### Hessian

In [ ]:
md('$$', PotentialDerivatives[2], '$$')

### Derivatives test

In [ ]:
def test_derivatives():
    args = testcase1()
    
    md('- Potential: $', potential_derivatives[0](*args), '$')
    md('- Gradient: $', potential_derivatives[1](*args), '$')
    md('- Hessian: $', potential_derivatives[2](*args), '$')

test_derivatives()

### Minimization test

In [ ]:
def test_minimize():
    def wrap(f):
        return lambda xs: f(np.array(xs, dtype=np.float))
    
    ni, Ki, Li, Pi, p = testcase1()
    args = [ni, Ki, Li, Pi]
    md('$n_i:', ni, ', K_i:', Ki, ', L_i:', Li, ', P_i:', Pi, ', p:', p, '$')
    
    f, g, h = [wrap(partial(f, *args)) for f in potential_derivatives]
    
    res = minimize(f, p.tolist(), jac=lambda x: g(x).flatten(), hess=h, method='trust-ncg')

    print(res)

test_minimize()